# **Libraries and other things**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!pip install ultralytics opencv-python torch torchvision torchaudio

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import os
import cv2
import glob
import shutil
import torch
import zipfile
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# **Dataset Loading**

In [ ]:
dataset_path = "/content/drive/MyDrive/Car Theft Model/Dataset"

In [ ]:
train_img_dir = os.path.join(dataset_path, "train", "images")
train_lbl_dir = os.path.join(dataset_path, "train", "labels")
val_img_dir = os.path.join(dataset_path, "valid", "images")
val_lbl_dir = os.path.join(dataset_path, "valid", "labels")
test_img_dir = os.path.join(dataset_path, "test", "images")
test_lbl_dir = os.path.join(dataset_path, "test", "labels")
yaml_path = os.path.join(dataset_path, "data.yaml")
print(train_img_dir)
print(train_lbl_dir)
print(val_img_dir)
print(val_lbl_dir)
print(test_img_dir)
print(test_lbl_dir)
print(yaml_path)

/content/drive/MyDrive/Car Theft Model/Dataset/train/images
/content/drive/MyDrive/Car Theft Model/Dataset/train/labels
/content/drive/MyDrive/Car Theft Model/Dataset/valid/images
/content/drive/MyDrive/Car Theft Model/Dataset/valid/labels
/content/drive/MyDrive/Car Theft Model/Dataset/test/images
/content/drive/MyDrive/Car Theft Model/Dataset/test/labels
/content/drive/MyDrive/Car Theft Model/Dataset/data.yaml


# **Preprocessing**

In [ ]:
base_dir = "/content/drive/MyDrive/Car Theft Model/Dataset"


for folder in ["train", "test", "valid"]:
    labels_path = os.path.join(base_dir, folder, "labels")

    if not os.path.exists(labels_path):
        print(f"🚨 Path does not exist: {labels_path}")
        continue

    # Rename files
    for filename in os.listdir(labels_path):
        file_path = os.path.join(labels_path, filename)


        if not filename.endswith(".txt"):
            new_filename = filename + ".txt"
            new_file_path = os.path.join(labels_path, new_filename)

            try:
                os.rename(file_path, new_file_path)
                print(f"✅ Renamed: {filename} → {new_filename}")
            except Exception as e:
                print(f"⚠️ Error renaming {filename}: {e}")

print("🎉 Renaming complete!")

🎉 Renaming complete!


In [ ]:
# 640 x 640
def resize_images(image_folder, target_size=(640, 640)):
    image_files = glob.glob(f"{image_folder}/*.jpg")
    total_images = len(image_files)
    for idx, img_file in enumerate(image_files):
        img = cv2.imread(img_file)
        img = cv2.resize(img, target_size)
        cv2.imwrite(img_file, img)
        print(f"Processed {idx+1}/{total_images} images in {image_folder}")

resize_images(train_img_dir)
resize_images(val_img_dir)
resize_images(test_img_dir)

Processed 1/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 2/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 3/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 4/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 5/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 6/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 7/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 8/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 9/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 10/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 11/848 images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images
Processed 12/848 images in /content/drive/MyDrive/Co

In [ ]:
def match_files(img_dir, lbl_dir):
  img_files = set([os.path.splitext(os.path.basename(f))[0] for f in glob.glob(os.path.join(img_dir, "*.jpg"))])
  lbl_files = set([os.path.splitext(os.path.basename(f))[0] for f in glob.glob(os.path.join(lbl_dir, "*.txt"))])

  unmatched_images = img_files - lbl_files
  unmatched_labels = lbl_files - img_files

  print(f"Unmatched images in {img_dir}: {len(unmatched_images)}")
  print(f"Unmatched labels in {lbl_dir}: {len(unmatched_labels)}")

  return unmatched_images, unmatched_labels



train_unmatched_images, train_unmatched_labels = match_files(train_img_dir, train_lbl_dir)
val_unmatched_images, val_unmatched_labels = match_files(val_img_dir, val_lbl_dir)
test_unmatched_images, test_unmatched_labels = match_files(test_img_dir, test_lbl_dir)



Unmatched images in /content/drive/MyDrive/Colab Notebooks/Dataset/train/images: 0
Unmatched labels in /content/drive/MyDrive/Colab Notebooks/Dataset/train/labels: 0
Unmatched images in /content/drive/MyDrive/Colab Notebooks/Dataset/valid/images: 0
Unmatched labels in /content/drive/MyDrive/Colab Notebooks/Dataset/valid/labels: 0
Unmatched images in /content/drive/MyDrive/Colab Notebooks/Dataset/test/images: 0
Unmatched labels in /content/drive/MyDrive/Colab Notebooks/Dataset/test/labels: 0


In [ ]:
def calculate_mismatch_percentage(img_dir, lbl_dir):
  img_files = set([os.path.splitext(os.path.basename(f))[0] for f in glob.glob(os.path.join(img_dir, "*.jpg"))])
  lbl_files = set([os.path.splitext(os.path.basename(f))[0] for f in glob.glob(os.path.join(lbl_dir, "*.txt"))])

  total_files = len(img_files)
  if total_files == 0:
    return 0.0

  unmatched_count = len(img_files - lbl_files) + len(lbl_files - img_files)
  mismatch_percentage = (unmatched_count / total_files) * 100

  return mismatch_percentage


train_mismatch_percentage = calculate_mismatch_percentage(train_img_dir, train_lbl_dir)
val_mismatch_percentage = calculate_mismatch_percentage(val_img_dir, val_lbl_dir)
test_mismatch_percentage = calculate_mismatch_percentage(test_img_dir, test_lbl_dir)


print(f"Train set mismatch percentage: {train_mismatch_percentage:.2f}%")
print(f"Validation set mismatch percentage: {val_mismatch_percentage:.2f}%")
print(f"Test set mismatch percentage: {test_mismatch_percentage:.2f}%")


Train set mismatch percentage: 0.00%
Validation set mismatch percentage: 0.00%
Test set mismatch percentage: 0.00%


In [ ]:
labels_folder = "/content/drive/MyDrive/Car Theft Model/Dataset/valid/labels"

def clean_bom_from_labels(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)

            with open(file_path, "r", encoding="utf-8-sig") as f:
                lines = f.readlines()

            with open(file_path, "w", encoding="utf-8") as f:
                f.writelines(lines)

            print(f"✅ Cleaned BOM from: {filename}")

clean_bom_from_labels(labels_folder)
print("🚀 All label files are cleaned and ready!")

✅ Cleaned BOM from: fdbb92e8-dcac-11ef-9634-0242ac1c000c_jpg.rf.7d9d6b812e157dc4d6ca96eead2f6777.txt
✅ Cleaned BOM from: ecc52cc4-dcac-11ef-9634-0242ac1c000c_jpg.rf.d0dc17c9ea95960aa63fb564881eb0a2.txt
✅ Cleaned BOM from: fde6b16c-dcac-11ef-9634-0242ac1c000c_jpg.rf.12c1306c11f7c52cbda3a561532089cc.txt
✅ Cleaned BOM from: e5a65b52-dcac-11ef-9634-0242ac1c000c_jpg.rf.33339e26b3321c94ade0fd4e7229b307.txt
✅ Cleaned BOM from: e5ae7b70-dcac-11ef-9634-0242ac1c000c_jpg.rf.a291c4011e2f3e7a297efd7884ce0074.txt
✅ Cleaned BOM from: dd29f3fc-dcad-11ef-9634-0242ac1c000c_jpg.rf.4e7955b1a5fe93831f109bb3553666c6.txt
✅ Cleaned BOM from: d83c5b64-dcad-11ef-9634-0242ac1c000c_jpg.rf.3149880b1abb4328d2e67d5bf6b93186.txt
✅ Cleaned BOM from: dd5bdade-dcad-11ef-9634-0242ac1c000c_jpg.rf.9e4aaafd2d77e2d942d1b937d2a00157.txt
✅ Cleaned BOM from: ecaaf174-dcac-11ef-9634-0242ac1c000c_jpg.rf.5b075f20ec9cf1e386bff5139978f116.txt
✅ Cleaned BOM from: e5be7674-dcac-11ef-9634-0242ac1c000c_jpg.rf.496087c4f695bf850cef15ea843

# **Loading Model**

In [ ]:
model = YOLO("yolov8n.pt")

# **Training**

In [ ]:
import csv
import time
import matplotlib.pyplot as plt
import numpy as np

log_file = "/content/drive/MyDrive/Car Theft Model/Logs/Accuracy Logs.csv"

with open(log_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Box Loss", "Class Loss", "DFL Loss", "mAP50", "mAP50-95"])

# Training Config
results = model.train(
    data=yaml_path,
    epochs=50,
    batch=32,
    imgsz=640,
    lr0=0.001,
    weight_decay=0.0005,
    momentum=0.937,
    augment=True
)

print("Available attributes in results:", dir(results))
if hasattr(results, 'results_dict'):
    print("Keys in results_dict:", results.results_dict.keys())

# Training History Retrieval
try:
    if hasattr(results, 'training_results'):
        print("Using training_results attribute")
        for epoch, data in enumerate(results.training_results, 1):
            box_loss = data.get('box_loss', 0)
            cls_loss = data.get('cls_loss', 0)
            dfl_loss = data.get('dfl_loss', 0)
            map50 = data.get('map50', 0)
            map50_95 = data.get('map', 0)

            with open(log_file, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([epoch, box_loss, cls_loss, dfl_loss, map50, map50_95])

            print(f"Logged epoch {epoch} metrics")

    elif hasattr(results, 'save_dir'):
        print("Using CSV files from save directory")
        import pandas as pd
        results_csv = str(results.save_dir / 'results.csv')

        if os.path.exists(results_csv):
            df = pd.read_csv(results_csv)
            print(f"Columns in results.csv: {df.columns.tolist()}")

            for index, row in df.iterrows():
                epoch = row.get('epoch', index + 1)
                box_loss = row.get('box_loss', 0)
                cls_loss = row.get('cls_loss', 0)
                dfl_loss = row.get('dfl_loss', 0)
                map50 = row.get('metrics/mAP50(B)', 0)
                map50_95 = row.get('metrics/mAP50-95(B)', 0)

                with open(log_file, mode="a", newline="") as file:
                    writer = csv.writer(file)
                    writer.writerow([epoch, box_loss, cls_loss, dfl_loss, map50, map50_95])

                print(f"Logged epoch {epoch} metrics from results.csv")

    else:
        print("Using final results dict")
        results_dict = results.results_dict
        print(f"Keys in results_dict: {results_dict.keys()}")

        for epoch in range(1, 3):
            map50 = results_dict.get('metrics/mAP50(B)', 0)
            map50_95 = results_dict.get('metrics/mAP50-95(B)', 0)

            box_loss = 0
            cls_loss = 0
            dfl_loss = 0

            with open(log_file, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([epoch, box_loss, cls_loss, dfl_loss, map50, map50_95])

            print(f"Logged epoch {epoch} metrics (final values)")

except Exception as e:
    print(f"Error extracting training logs: {str(e)}")

    print("\nDumping all available information for debugging:")
    for attr in dir(results):
        if not attr.startswith('__'):
            try:
                value = getattr(results, attr)
                print(f"{attr}: {type(value)}")
                if isinstance(value, dict):
                    print(f"  Dictionary keys: {value.keys()}")
            except:
                print(f"{attr}: Unable to access")

model.save("/content/drive/MyDrive/Car Theft Model/Models/yolov8n_75e_final.pt")
print("Model saved successfully!")

Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Colab Notebooks/yolov8n_75e_f.pt, data=/content/drive/MyDrive/Colab Notebooks/Dataset/data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning /content/drive/.shortcut-targets-by-id/1osQbHLpARn1Inx8Jbr8gzOCF4mRK3u3v/Colab Notebooks/Dataset/train/labels.cache... 848 images, 15 backgrounds, 0 corrupt: 100%|██████████| 848/848 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/.shortcut-targets-by-id/1osQbHLpARn1Inx8Jbr8gzOCF4mRK3u3v/Colab Notebooks/Dataset/valid/labels.cache... 81 images, 2 backgrounds, 0 corrupt: 100%|██████████| 81/81 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      5.09G     0.4503     0.2511      1.034         15        640: 100%|██████████| 27/27 [00:17<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.43it/s]

                   all         81         79      0.937      0.962      0.962      0.818



50 epochs completed in 0.292 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]


                   all         81         79      0.951      0.975      0.972      0.852
Speed: 0.3ms preprocess, 8.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train3
Available attributes in results: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'ap_class_index', 'box', 'class_result', 'confusion_matrix', 'curves', 'curves_results', 'fitness', 'keys', 'maps', 'mean_results', 'names', 'plot', 'process', 'results_dict', 'save_dir', 'speed', 'task']
Keys in results_dict: dict_keys(['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'fitness'])
Using CSV files from save directory
Columns in

# **Visualization**

In [25]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.signal import savgol_filter

log_file = "/content/drive/MyDrive/Car Theft Model/Logs/Accuracy Logs.csv"
df = pd.read_csv(log_file)

df['training_progress'] = df.index / len(df) * 100
df['convergence_ratio'] = df['mAP50-95'] / df['mAP50']
df['plateau_detection'] = df['mAP50'].diff().rolling(window=5).mean()

df['mAP50_smooth'] = savgol_filter(df['mAP50'], window_length=7, polyorder=2) if len(df) > 7 else df['mAP50']
df['mAP50-95_smooth'] = savgol_filter(df['mAP50-95'], window_length=7, polyorder=2) if len(df) > 7 else df['mAP50-95']


fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Primary Metrics Performance",
        "Performance Improvement from Baseline",
        "Convergence Analysis",
        "Learning Stability"
    ),
    specs=[
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "scatter"}, {"type": "scatter"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

colors = {
    'mAP50': '#1f77b4',         # Blue
    'mAP50-95': '#ff7f0e',      # Orange
    'background': '#F8F9FA',    # Light gray background
    'grid': '#E4E4E4',          # Grid color
    'annotation': '#2C3E50',    # Dark blue-gray for annotations
    'trend': '#2ecc71',         # Green for trends
    'area': 'rgba(52, 152, 219, 0.2)'  # Light blue for area
}

expected_min = 0.85
expected_max = 0.95

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=[expected_max] * len(df),
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=[expected_min] * len(df),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(144, 238, 144, 0.2)',  # Light green
        showlegend=True,
        name="Expected Range",
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50"],
        mode="lines+markers",
        name="mAP50",
        line=dict(color=colors['mAP50'], width=2.5, shape='spline'),
        marker=dict(symbol="triangle-up", size=9, color=colors['mAP50'], line=dict(width=1, color='white')),
        hovertemplate="<b>Epoch:</b> %{x}<br><b>mAP50:</b> %{y:.4f}"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50-95"],
        mode="lines+markers",
        name="mAP50-95",
        line=dict(color=colors['mAP50-95'], width=2.5, shape='spline'),
        marker=dict(symbol="x", size=9, color=colors['mAP50-95'], line=dict(width=1, color='white')),
        hovertemplate="<b>Epoch:</b> %{x}<br><b>mAP50-95:</b> %{y:.4f}"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50_smooth"],
        mode="lines",
        line=dict(color=colors['mAP50'], width=1.5, dash='dot'),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50-95_smooth"],
        mode="lines",
        line=dict(color=colors['mAP50-95'], width=1.5, dash='dot'),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

df['mAP50_rel_improvement'] = (df['mAP50'] / df['mAP50'].iloc[0] - 1) * 100
df['mAP50-95_rel_improvement'] = (df['mAP50-95'] / df['mAP50-95'].iloc[0] - 1) * 100

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50_rel_improvement"],
        mode="lines",
        name="mAP50 Improvement",
        line=dict(color=colors['mAP50'], width=2, shape='spline'),
        fill='tozeroy',
        fillcolor=f'rgba(31, 119, 180, 0.3)',
        hovertemplate="<b>Epoch:</b> %{x}<br><b>Improvement:</b> %{y:.2f}%"
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50-95_rel_improvement"],
        mode="lines",
        name="mAP50-95 Improvement",
        line=dict(color=colors['mAP50-95'], width=2, shape='spline'),
        fill='tozeroy',
        fillcolor=f'rgba(255, 127, 14, 0.3)',
        hovertemplate="<b>Epoch:</b> %{x}<br><b>Improvement:</b> %{y:.2f}%"
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["convergence_ratio"],
        mode="lines+markers",
        name="Convergence Ratio",
        line=dict(color="#9b59b6", width=2.5, shape='spline'),  # Purple
        marker=dict(size=7, symbol="circle", color="#9b59b6", line=dict(width=1, color='white')),
        hovertemplate="<b>Epoch:</b> %{x}<br><b>mAP50-95/mAP50:</b> %{y:.4f}"
    ),
    row=2, col=1
)

ideal_convergence = 1.0
fig.add_trace(
    go.Scatter(
        x=[df["Epoch"].min(), df["Epoch"].max()],
        y=[ideal_convergence, ideal_convergence],
        mode="lines",
        line=dict(color="rgba(155, 89, 182, 0.5)", width=1.5, dash='dash'),
        name="Ideal Convergence",
        hoverinfo='skip'
    ),
    row=2, col=1
)

df['mAP50_change'] = df['mAP50'].diff().fillna(0)
df['mAP50-95_change'] = df['mAP50-95'].diff().fillna(0)

df['mAP50_change_smooth'] = savgol_filter(df['mAP50_change'], window_length=5, polyorder=1) if len(df) > 5 else df['mAP50_change']
df['mAP50-95_change_smooth'] = savgol_filter(df['mAP50-95_change'], window_length=5, polyorder=1) if len(df) > 5 else df['mAP50-95_change']

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50_change_smooth"],
        mode="lines",
        name="mAP50 Rate of Change",
        line=dict(color=colors['mAP50'], width=2, shape='spline'),
        hovertemplate="<b>Epoch:</b> %{x}<br><b>Change:</b> %{y:.4f}"
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=df["Epoch"],
        y=df["mAP50-95_change_smooth"],
        mode="lines",
        name="mAP50-95 Rate of Change",
        line=dict(color=colors['mAP50-95'], width=2, shape='spline'),
        hovertemplate="<b>Epoch:</b> %{x}<br><b>Change:</b> %{y:.4f}"
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=[df["Epoch"].min(), df["Epoch"].max()],
        y=[0, 0],
        mode="lines",
        line=dict(color="rgba(0, 0, 0, 0.3)", width=1.5, dash='dash'),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=2, col=2
)

# ANNOTATIONS
best_map50_epoch = df['mAP50'].idxmax()
best_map50_95_epoch = df['mAP50-95'].idxmax()
first_plateau_epoch = df[df['plateau_detection'].abs() < 0.001].index[3] if len(df[df['plateau_detection'].abs() < 0.001]) > 3 else None
highest_improvement_epoch = df['mAP50-95_rel_improvement'].idxmax()

fig.add_annotation(
    x=df['Epoch'].iloc[best_map50_epoch],
    y=df['mAP50'].iloc[best_map50_epoch],
    text=f"Best mAP50: {df['mAP50'].iloc[best_map50_epoch]:.4f}",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor=colors['mAP50'],
    font=dict(color="white", size=11),
    bordercolor=colors['mAP50'],
    bgcolor=colors['mAP50'],
    borderwidth=1,
    borderpad=4,
    row=1, col=1
)

fig.add_annotation(
    x=df['Epoch'].iloc[best_map50_95_epoch],
    y=df['mAP50-95'].iloc[best_map50_95_epoch],
    text=f"Best mAP50-95: {df['mAP50-95'].iloc[best_map50_95_epoch]:.4f}",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor=colors['mAP50-95'],
    font=dict(color="white", size=11),
    bordercolor=colors['mAP50-95'],
    bgcolor=colors['mAP50-95'],
    borderwidth=1,
    borderpad=4,
    row=1, col=1
)

fig.add_annotation(
    x=df['Epoch'].iloc[highest_improvement_epoch],
    y=df['mAP50-95_rel_improvement'].iloc[highest_improvement_epoch],
    text=f"Highest Improvement: {df['mAP50-95_rel_improvement'].iloc[highest_improvement_epoch]:.1f}%",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor=colors['mAP50-95'],
    font=dict(color="white", size=11),
    bordercolor=colors['mAP50-95'],
    bgcolor=colors['mAP50-95'],
    borderwidth=1,
    borderpad=4,
    row=1, col=2
)

if first_plateau_epoch:
    fig.add_annotation(
        x=df['Epoch'].iloc[first_plateau_epoch],
        y=df['mAP50_change_smooth'].iloc[first_plateau_epoch],
        text="Training Plateau Detected",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#e74c3c",  # Red
        font=dict(color="white", size=11),
        bordercolor="#e74c3c",
        bgcolor="#e74c3c",
        borderwidth=1,
        borderpad=4,
        row=2, col=2
    )

final_map50 = df['mAP50'].iloc[-1]
final_map50_95 = df['mAP50-95'].iloc[-1]
evaluation_text = (
    f"<b>Model Performance Summary</b><br>" +
    f"Final mAP50: {final_map50:.4f}<br>" +
    f"Final mAP50-95: {final_map50_95:.4f}<br>" +
    f"Improvement: {df['mAP50_rel_improvement'].iloc[-1]:.1f}% & {df['mAP50-95_rel_improvement'].iloc[-1]:.1f}%<br>" +
    f"Convergence: {(final_map50_95/final_map50*100):.1f}%"
)

fig.add_annotation(
    x=0.5, y=1.18,
    xref="paper", yref="paper",
    text=evaluation_text,
    showarrow=False,
    font=dict(size=14),
    align="center",
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="#2C3E50",
    borderwidth=1,
    borderpad=6
)

fig.update_layout(
    title=dict(
        text="Model's Performance Analytics During Training(Nano)",
        font=dict(size=22, color="#2C3E50", family="Arial, sans-serif"),
        x=0.5,
        y=0.98
    ),
    template="plotly_white",
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background'],
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="#E4E4E4",
        borderwidth=1,
        font=dict(family="Arial, sans-serif", size=12, color="#2C3E50")
    ),
    height=900,
    width=1200,
    margin=dict(l=80, r=80, t=180, b=100),
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial, sans-serif"
    )
)

for row in [1, 2]:
    for col in [1, 2]:
        fig.update_xaxes(
            title_text="Epochs",
            row=row, col=col,
            gridcolor=colors['grid'],
            title_font=dict(size=13, family="Arial, sans-serif", color="#2C3E50"),
            tickfont=dict(size=11, family="Arial, sans-serif", color="#2C3E50"),
            showline=True,
            linewidth=2,
            linecolor=colors['grid'],
            zeroline=False
        )

        fig.update_yaxes(
            gridcolor=colors['grid'],
            title_font=dict(size=13, family="Arial, sans-serif", color="#2C3E50"),
            tickfont=dict(size=11, family="Arial, sans-serif", color="#2C3E50"),
            showline=True,
            linewidth=2,
            linecolor=colors['grid'],
            zeroline=False
        )

fig.update_yaxes(title_text="Performance (mAP)", row=1, col=1, range=[0.65, 1.0])
fig.update_yaxes(title_text="Improvement from Baseline (%)", row=1, col=2)
fig.update_yaxes(title_text="Convergence Ratio (mAP50-95/mAP50)", row=2, col=1)
fig.update_yaxes(title_text="Rate of Change", row=2, col=2)

fig.show()


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


# **Testing**

In [ ]:
from google.colab.patches import cv2_imshow
import torch
import numpy as np
from ultralytics import YOLO
import cv2
import time


model = YOLO("/content/drive/MyDrive/Car Theft Model/Models/yolov8n_50e_final.pt")


video_path = "/content/drive/MyDrive/Car Theft Model/Test Inputs"
cap = cv2.VideoCapture(video_path)


fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


output_path = "/content/drive/MyDrive/Car Theft Model/Test Inputs/demo_output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
###
CONFIDENCE_THRESHOLD = 0.88

def create_gradient_box(img, x1, y1, x2, y2, color, thickness=2, alpha=0.3):
    overlay = img.copy()
    cv2.rectangle(overlay, (int(x1), int(y1)), (int(x2), int(y2)), color, -1)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
    return img


start_time = time.time()


COLORS = {
    'high_conf': (0, 0, 255),    # Red for high confidence
    'med_conf': (0, 165, 255),   # Orange for medium confidence
    'low_conf': (0, 255, 255),   # Yellow for low confidence
    'box_highlight': (255, 255, 255)  # White for box highlight
}


frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    frame_resized = cv2.resize(frame, (640, 640)) if (frame.shape[:2] != (640, 640)) else frame

    results = model(frame_resized)

    detections = results[0].boxes.data

    elapsed_time = time.time() - start_time
    mins, secs = divmod(elapsed_time, 60)
    timestamp = f"{int(mins):02d}:{int(secs):02d}"

    header_height = 40
    header = np.zeros((header_height, frame_resized.shape[1], 3), dtype=np.uint8)

    for i in range(header_height):
        alpha = 1.0 - (i / header_height)
        header[i, :] = (50, 50, 50 + int(alpha * 50))

    cv2.putText(header, f"Time: {timestamp}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.putText(header, f"Frame: {frame_count}", (frame_resized.shape[1] - 150, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    frame_with_header = np.vstack([header, frame_resized])
    frame_with_header = cv2.resize(frame_with_header, (frame_resized.shape[1], frame_resized.shape[0]))
    frame_resized = frame_with_header

    for det in detections:
        x1, y1, x2, y2, conf, cls = det.tolist()
        class_id = int(cls)

        if conf < CONFIDENCE_THRESHOLD:
            continue

        if conf > 0.8:
            color = COLORS['high_conf']
            risk_level = "HIGH RISK"
        elif conf > 0.65:
            color = COLORS['med_conf']
            risk_level = "MEDIUM RISK"
        else:
            color = COLORS['low_conf']
            risk_level = "LOW RISK"

        label = f"{model.names[class_id]}: {conf:.2f}"

        frame_resized = create_gradient_box(frame_resized, x1, y1, x2, y2, color, thickness=2)

        corner_length = min(30, int((x2-x1) * 0.3))

        cv2.line(frame_resized, (int(x1), int(y1)), (int(x1 + corner_length), int(y1)), COLORS['box_highlight'], 3)
        cv2.line(frame_resized, (int(x1), int(y1)), (int(x1), int(y1 + corner_length)), COLORS['box_highlight'], 3)

        cv2.line(frame_resized, (int(x2), int(y1)), (int(x2 - corner_length), int(y1)), COLORS['box_highlight'], 3)
        cv2.line(frame_resized, (int(x2), int(y1)), (int(x2), int(y1 + corner_length)), COLORS['box_highlight'], 3)

        cv2.line(frame_resized, (int(x1), int(y2)), (int(x1 + corner_length), int(y2)), COLORS['box_highlight'], 3)
        cv2.line(frame_resized, (int(x1), int(y2)), (int(x1), int(y2 - corner_length)), COLORS['box_highlight'], 3)

        cv2.line(frame_resized, (int(x2), int(y2)), (int(x2 - corner_length), int(y2)), COLORS['box_highlight'], 3)
        cv2.line(frame_resized, (int(x2), int(y2)), (int(x2), int(y2 - corner_length)), COLORS['box_highlight'], 3)

        cv2.line(frame_resized, (int(x1), int(y1)), (int(x1 + corner_length), int(y1)), color, 2)
        cv2.line(frame_resized, (int(x1), int(y1)), (int(x1), int(y1 + corner_length)), color, 2)
        cv2.line(frame_resized, (int(x2), int(y1)), (int(x2 - corner_length), int(y1)), color, 2)
        cv2.line(frame_resized, (int(x2), int(y1)), (int(x2), int(y1 + corner_length)), color, 2)
        cv2.line(frame_resized, (int(x1), int(y2)), (int(x1 + corner_length), int(y2)), color, 2)
        cv2.line(frame_resized, (int(x1), int(y2)), (int(x1), int(y2 - corner_length)), color, 2)
        cv2.line(frame_resized, (int(x2), int(y2)), (int(x2 - corner_length), int(y2)), color, 2)
        cv2.line(frame_resized, (int(x2), int(y2)), (int(x2), int(y2 - corner_length)), color, 2)

        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        text_width = text_size[0] + 10

        label_y1 = max(0, int(y1) - 42)
        label_y2 = max(0, int(y1) - 10)

        overlay = frame_resized.copy()
        cv2.rectangle(overlay, (int(x1), label_y1), (int(x1) + text_width, label_y2), color, -1)
        cv2.addWeighted(overlay, 0.8, frame_resized, 0.2, 0, frame_resized)

        conf_label = f"Confidence: {conf:.2f}"
        conf_text_size = cv2.getTextSize(conf_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        conf_width = conf_text_size[0] + 10

        risk_text_size = cv2.getTextSize(risk_level, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        risk_width = risk_text_size[0] + 10

        risk_y1 = int(y2) + 10
        risk_y2 = int(y2) + 30

        overlay = frame_resized.copy()
        cv2.rectangle(overlay, (int(x1), risk_y1), (int(x1) + risk_width, risk_y2), color, -1)
        cv2.addWeighted(overlay, 0.8, frame_resized, 0.2, 0, frame_resized)

        cv2.putText(frame_resized, label, (int(x1) + 5, label_y2 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame_resized, risk_level, (int(x1) + 5, risk_y2 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        bar_width = int((x2 - x1) * 0.8)
        bar_height = 5
        bar_x = int(x1)
        bar_y = int(y2) + 35

        cv2.rectangle(frame_resized, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (100, 100, 100), -1)

        filled_width = int(bar_width * conf)
        cv2.rectangle(frame_resized, (bar_x, bar_y), (bar_x + filled_width, bar_y + bar_height), color, -1)

    annotated_frame = cv2.resize(frame_resized, (frame_width, frame_height))

    out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_path}")


0: 640x640 (no detections), 8.9ms
Speed: 2.1ms preprocess, 8.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.2ms preprocess, 7.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.2ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.2ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.5ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.4ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.1ms preprocess, 7.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.2ms preprocess, 7.4ms inference, 0.5ms 